<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_03_Filter_Images_by_Collections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to ingest several images and add them to different collections. We will then filter contents by the collections.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [ ]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [ ]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [ ]:
from typing import List, Optional

async def ingest_uri(uri: str, collection_id: str):
    if graphlit.client is None:
        return;

    try:
        # Using synchronous mode, so the notebook waits for the content to be ingested
        response = await graphlit.client.ingest_uri(uri=uri, collections=[input_types.EntityReferenceInput(id=collection_id)], is_synchronous=True)

        return response.ingest_uri.id if response.ingest_uri is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def create_collection(name: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.create_collection(
            collection=input_types.CollectionInput(
                name=name
            )
        )

        return response.create_collection.id if response.create_collection is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def query_contents(collection_ids: List[str]):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.query_contents(
            filter=input_types.ContentFilter(
                collections=[input_types.EntityReferenceFilter(id=collection_id) for collection_id in collection_ids]
            )
        )

        return response.contents.results if response.contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)

async def delete_all_collections():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_collections(is_synchronous=True)

Execute Graphlit example

In [ ]:
from IPython.display import display, Markdown

# Remove any existing contents and collections; only needed for notebook example
await delete_all_contents()
await delete_all_collections()

print('Deleted all contents and collections.')

collection_one_id = await create_collection("Collection One")
collection_two_id = await create_collection("Collection Two")

if collection_one_id is not None and collection_two_id is not None:
    await ingest_uri(uri="https://graphlitplatform.blob.core.windows.net/samples/graphlit-logo-white-text.png", collection_id=collection_one_id)
    await ingest_uri(uri="https://graphlitplatform.blob.core.windows.net/samples/graphlit-logo.svg", collection_id=collection_one_id)
    await ingest_uri(uri="https://graphlitplatform.blob.core.windows.net/samples/graphlit-logo-transparent-512.png", collection_id=collection_two_id)

    contents = await query_contents(collection_ids=[collection_one_id])

    if contents is not None:
        for content in contents:
            if content is not None:
                print(f"From Collection One: {content.name}")

    contents = await query_contents(collection_ids=[collection_two_id])

    if contents is not None:
        for content in contents:
            if content is not None:
                print(f"From Collection Two: {content.name}")

    contents = await query_contents(collection_ids=[collection_one_id,collection_two_id])

    if contents is not None:
        for content in contents:
            if content is not None:
                print(f"From both collections: {content.name}")
